# Project: Micromouse simulation to explore common planning algorithms

In this project, we are given a 2D grid world representing the maze, and a 'mouse' robot that needs to navigate from a start to goal configuration.

In this code, we will explore A*.

The workspace is a 200x200 circular maze, with 0s representing free space and 1s representing obstacles.

The mouse is represented by a 3D array showing different orientations

Our goal is to:
- Analyze the workspace and mouse configurations
- Construct the configuration space (C-space)
- Plan a path for the rod from start to goal using A* search

To accomplish this, we are given:
- Environment map (200x200 grid)
- Utility functions for visualization

We will process the data, construct the C-space, run A* search, and visualize the results.

## Imports and Helper functions

In [1]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math
from PIL import Image

def load_and_process_environment(image_path):
    img = Image.open(image_path).convert('L')
    img = img.resize((200, 200))  # Resize to 200 x 200
    img_array = np.array(img)
    img_array = img_array < 200  # Convert to binary image
    return img_array.astype(np.int)

def create_c_space(env_map, mouse_shape, orientation_steps):
    max_angle = 360
    c_space = np.zeros((env_map.shape[0], env_map.shape[1], max_angle // orientation_steps))
    for i in range(max_angle // orientation_steps):
        rotated_mouse = mouse_shape[:, :, i]
        convolved = signal.convolve2d(env_map, np.flipud(np.fliplr(rotated_mouse)), mode='same', boundary='fill', fillvalue=1)
        c_space[:, :, i] = convolved > 0
    return c_space.astype(np.int)

def normalize_image(img: np.ndarray):
    return (img > 0).astype(np.int)

def plot_enviroment(img: np.ndarray, obj: np.ndarray, state: tuple):
    orientation = state[2] % 12
    dims = obj.shape
    dim_x = int((dims[0] - 1) / 2)
    dim_y = int((dims[1] - 1) / 2)
    start_x = max(state[0] - dim_x, 0)
    end_x = min(state[0] + dim_x + 1, img.shape[0])
    start_y = max(state[1] - dim_y, 0)
    end_y = min(state[1] + dim_y + 1, img.shape[1])
    merged_img = np.copy(img).astype(float)
    # Adjusting the slices of the mouse object to match the environment slice
    obj_slice_x = max(0, dim_x - state[0])
    obj_slice_y = max(0, dim_y - state[1])
    obj_slice_end_x = obj_slice_x + (end_x - start_x)
    obj_slice_end_y = obj_slice_y + (end_y - start_y)
    merged_img[start_x:end_x, start_y:end_y] += obj[obj_slice_x:obj_slice_end_x, obj_slice_y:obj_slice_end_y, orientation] * 0.5
    return merged_img

def plotting_results(environment: np.ndarray, mouse: np.ndarray, path: list, save_path: str):
    fig = plt.figure()
    imgs = []
    for state in path:
        im = plot_enviroment(environment, mouse, state)
        imgs.append([plt.imshow(im)])
    ani = animation.ArtistAnimation(fig, imgs, interval=50, blit=True)
    ani.save(save_path)
    plt.show()

def create_mouse_shape(orientation_steps):
    max_angle = 360
    mouse_shape = np.zeros((11, 11, max_angle // orientation_steps), dtype=np.int)
    center = (5, 5)
    length = 10
    for i in range(max_angle // orientation_steps):
        angle = math.radians(i * orientation_steps)
        for l in range(-length, length + 1):
            x = int(center[0] + l * math.cos(angle))
            y = int(center[1] + l * math.sin(angle))
            if 0 <= x < 11 and 0 <= y < 11:
                mouse_shape[x, y, i] = 1
    return mouse_shape

def is_free(node, c_space):
    x, y, theta = node
    if 0 <= x < c_space.shape[0] and 0 <= y < c_space.shape[1]:
        return c_space[x, y, theta] == 0
    return False

def get_successors_parametrized(node, c_space, movement_factor, orientation_step):
    successors = []
    x, y, theta = node
    # Adjust the orientation step based on the hyperparameter
    orientation_radians = math.radians(theta * orientation_step)
    cos_theta, sin_theta = math.cos(orientation_radians), math.sin(orientation_radians)

    # Adjust the successor generation based on movement factor
    for move in [-movement_factor, 0, movement_factor]:  # Backward, stay, forward
        new_x = x + int(round(cos_theta * move))
        new_y = y + int(round(sin_theta * move))
        for rotation in [-1, 0, 1]:  # Rotate left, no rotation, rotate right
            new_theta = (theta + rotation) % (360 // orientation_step)
            new_node = (new_x, new_y, new_theta)
            if is_free(new_node, c_space):
                successors.append(new_node)
    return successors

def reconstruct_path(came_from, current):
    total_path = [current]
    while current in came_from:
        current = came_from[current]
        total_path.append(current)
    return total_path[::-1]

def a_star(start, goal, c_space, heuristic, get_successors):
    open_set = {start: heuristic(start, goal)}
    came_from = {}
    g_score = {start: 0}
    f_score = {start: heuristic(start, goal)}
    visited_nodes = 0
    while open_set:
        current = min(open_set, key=open_set.get)
        if current[:2] == goal[:2]:  # Modified to compare (x, y) only
            path = reconstruct_path(came_from, current)
            return path, visited_nodes
        open_set.pop(current)
        for neighbor in get_successors(current, c_space):
            visited_nodes += 1
            tentative_g_score = g_score[current] + 1
            tentative_f_score = tentative_g_score + heuristic(neighbor, goal)
            if tentative_f_score < f_score.get(neighbor, float('inf')):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_f_score
                open_set[neighbor] = tentative_f_score
    return None, visited_nodes

def l1_heuristic(node, goal):
    x1, y1, _ = node
    x2, y2, _ = goal
    return abs(x1 - x2) + abs(y1 - y2)

def euclidean_heuristic(node, goal):
    x1, y1, _ = node
    x2, y2, _ = goal
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)    

def radial_heuristic(node, goal):
    x1, y1, _ = node
    x2, y2, _ = goal
    direct_distance = math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    maze_center = (100, 100)
    angle_to_goal = math.atan2(y2 - maze_center[1], x2 - maze_center[0])
    angle_to_node = math.atan2(y1 - maze_center[1], x1 - maze_center[0])
    angle_to_goal = angle_to_goal % (2 * math.pi)
    angle_to_node = angle_to_node % (2 * math.pi)
    angular_distance = min(abs(angle_to_goal - angle_to_node), 
                           2 * math.pi - abs(angle_to_goal - angle_to_node))
    average_radius = (math.sqrt((x1 - maze_center[0])**2 + (y1 - maze_center[1])**2) +
                      math.sqrt((x2 - maze_center[0])**2 + (y2 - maze_center[1])**2)) / 2
    circular_distance = angular_distance * average_radius
    return min(direct_distance, circular_distance)

## Visualizing Maze and Mouse Config

In [ ]:
environment2 = load_and_process_environment('maze3.jpg')
print("Environment2 shape:", environment2.shape)

In [ ]:
# Visualize Maze2
fig = plt.figure(figsize=(7,7)) 
plt.imshow(environment2, interpolation='none')
plt.title('Maze 2')
plt.show()

In [4]:
# Define start and goal configurations
start_config2 = (7, 105, 0)  # Maze 3
goal_config2 = (100, 100, 0)  # Maze 3

# Parameter search list
orientations = [10, 20, 30]  # degree intervals
movement_factors = [1, 2, 3]
heuristics = [l1_heuristic, euclidean_heuristic, radial_heuristic]

In [ ]:
# Consider random config of 45 deg intervals
mouse_shape = create_mouse_shape(45)

In [ ]:
# Plot mouse in start pos of maze2
env_obj = plot_enviroment(environment2, mouse_shape, start_config2)
plt.figure(figsize=(7,7))
plt.title('Environment with mouse in start')
plt.imshow(env_obj, interpolation='none')
plt.show()

In [ ]:
# Plot mouse in goal pos of maze2
env_obj = plot_enviroment(environment2, mouse_shape, goal_config2)
plt.figure(figsize=(7,7))
plt.title('Environment with mouse in goal')
plt.imshow(env_obj, interpolation='none')
plt.show()

## Constructing the C-Space

In [ ]:
# Normalize maze2 map
env_map2 = normalize_image(environment2)

# Create C-space
c_space = create_c_space(env_map2, mouse_shape, 45)
    
# Visualize the C-space slices for each orientation
fig, axs = plt.subplots(1, 8, figsize=(24, 6))
for i in range(8):
    axs[i].set_title('Orientation {}'.format(i*45))
    axs[i].imshow(c_space[:,:,i])
plt.show()

print("C-space shape:", c_space.shape)

## Start simulation for multiple parameters

### Maze 2

In [9]:
for orientation_step in orientations:
    mouse_shape = create_mouse_shape(orientation_step)
    c_space = create_c_space(env_map2, mouse_shape, orientation_step)

    for movement_factor in movement_factors:
        for heuristic in heuristics:
            get_successors_func = lambda node, c_space: get_successors_parametrized(
                node, c_space, movement_factor, orientation_step
            )
            print(f"Orientation Step: {orientation_step}, Movement Factor: {movement_factor}, Heuristic: {heuristic.__name__}")
            path, nodes_visit = a_star(
                start_config2, goal_config2, c_space, heuristic, get_successors_func
            )
            # Print the results for each combination
            if path:
                print("Final path cost:", len(path))
                print("Nodes visited:", nodes_visit)
            else:
                print("No path found.")
                print("Nodes visited:", nodes_visit)

/home/adithya/miniconda3/envs/dl/lib/python3.7/site-packages/ipykernel_launcher.py:57: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/adithya/miniconda3/envs/dl/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release no

Orientation Step: 10, Movement Factor: 1, Heuristic: l1_heuristic
Final path cost: 456
Nodes visited: 1765562
Orientation Step: 10, Movement Factor: 1, Heuristic: euclidean_heuristic
Final path cost: 453
Nodes visited: 1537000
Orientation Step: 10, Movement Factor: 1, Heuristic: radial_heuristic
Final path cost: 453
Nodes visited: 1737754
Orientation Step: 10, Movement Factor: 2, Heuristic: l1_heuristic
Final path cost: 275
Nodes visited: 3164386
Orientation Step: 10, Movement Factor: 2, Heuristic: euclidean_heuristic
Final path cost: 271
Nodes visited: 1733916
Orientation Step: 10, Movement Factor: 2, Heuristic: radial_heuristic
Final path cost: 263
Nodes visited: 2316394
Orientation Step: 10, Movement Factor: 3, Heuristic: l1_heuristic
Final path cost: 207
Nodes visited: 2666360
Orientation Step: 10, Movement Factor: 3, Heuristic: euclidean_heuristic
Final path cost: 208
Nodes visited: 2144594
Orientation Step: 10, Movement Factor: 3, Heuristic: radial_heuristic
Final path cost: 193


## Final Solution with optimal parameters

### Maze 2

In [ ]:
mouse_shape = create_mouse_shape(30)
env_map2 = normalize_image(environment2)
c_space = create_c_space(env_map2, mouse_shape, 30)

get_successors_func = lambda node, c_space: get_successors_parametrized(node, c_space, 3, 30)
path, nodes_visit = a_star(start_config2, goal_config2, c_space, radial_heuristic, get_successors_func)
plotting_results(environment2, mouse_shape, path, 'mouse_path_maze2.mp4')